<a href="https://colab.research.google.com/github/jac7az/A-Fool-s-Hand/blob/main/A_Fools_Hand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rules for A Fool's Hand


1.   Player starts with 5 cards in their hand
2.   There are three groups in the middle, each with 2 cards.
3. 1 game is made up of 4 rounds, with one pair of cards in the middle uncovered in each of the first three rounds.
4.    A combo is formed using the cards in the middle and your own cards. Each combo has a set number of points, listed below. Each player can place up to 4 cards in their zone for each group to complete a combo. Once the cards are placed and the round ends, the cards cannot be moved at all, not to another group or back to the hand. After all cards are set for the round, draw 2 cards.
5. During each round, players can discard up to 3 cards in their hand in exchange for new ones.
6.  At the end of the 4th round, points are added up for a final score. Multiple combinations in a single zone are counted.

# Combos


*   Straight Flush: 120 points
*   Four Knights: 80 points
* Flush: 65 points
* Straight: 60 points
* Full House: 55 points
* Twin Pairs: 30 points
* Three Sages: 25 points
* Two Stars: 15 points
* No Hand: 0 points



In [ ]:
import random
from itertools import product
from random import shuffle
from collections import Counter

In [ ]:
def create_deck():
  suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
  ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
  deck = list(product(suits,ranks))
  random.shuffle(deck)
  return deck

In [ ]:
def draw(num_cards,player_hand, deck):
  for card in range(num_cards):
    if len(deck)!=0:
      player_hand.append(deck.pop())

#input: which card would you like to discard
def discard(player_hand,trash,deck):
  num_discard=0
  while num_discard<3:
    print("Current hand: ", player_hand)
    try:
      answer=input("Enter the card position of your hand to discard or press 'c' to cancel: ").strip()
      if answer.lower()=='c':
        break
      elif 0<int(answer)<=len(player_hand):
        trash.append(player_hand.pop(int(answer)-1))
        draw(1,player_hand,deck)
        num_discard+=1
      else:
        print("Invalid input. Please try again")
    except ValueError:
      print("Invalid input. Please try again")

def play(player_hand, player_zones):
  while len(player_hand)>0:
    print(player_hand)
    card=input("Enter the position of the card you want to play: ").strip()
    zone=input("Enter the zone (1-3) to place it in: ").strip()
    try:
      card=int(card)
      zone= int(zone)
      if 0<=card-1<len(player_hand) and zone-1<3:
        if len(player_zones[zone-1])<=4:
          player_zones[zone-1].append(player_hand.pop(card-1))
        else:
          print("Maximum cards placed. Choose another zone.")
      else:
        print("Invalid input. Please try again.")
    except ValueError:
      print("Invalid input. Please try again.")

In [ ]:
def is_straight(ranks): #5 cards in sequential order
  if len(ranks)<5:
    return False
  rank_dict= {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8,
                '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13, 'A':14}
  in_order=[]
  for rank in ranks:
    in_order.append(rank_dict[rank])
  ##Ace=1
  if {14, 2, 3, 4, 5}.issubset(in_order):
    print("Straight")
    return 60
  ##All other straights, including Ace=14
  in_order.sort()
  straight_counter=1
  for i in range(len(in_order)-1):
    if in_order[i]+1==in_order[i+1]:
      straight_counter+=1
    elif in_order[i]==in_order[i+1]:
      pass
    else:
      straight_counter=1
    if straight_counter>=5:
      print("Straight")
      return 60
  return 0

def is_flush(suits): #When 5 suits are the same
  if len(suits)<5:
    return 0
  counts=Counter(suits)
  for count in counts.values():
    if count>=5:
      print("Flush")
      return 65
  return 0

# def is_straight_flush(suits,ranks): #5 cards in sequential order, all with the same suit
#   if is_flush(suits) and is_straight(ranks):
#     return 125
#   return 0

def is_four_knights(ranks): #4 of a kind
  if len(ranks)<4:
    return 0
  counts=Counter(ranks)
  for count in counts.values():
    if count>=4:
      return 80
  return 0

def is_full_house(ranks): #a pair + a triple
  if len(ranks)<5:
    return 0
  counts=Counter(ranks)
  if 3 in counts.values() and 2 in counts.values():
    print("Full House")
    return 55
  return 0

def three_sages(ranks): #a triple
  if len(ranks)<3:
    return 0
  counts=Counter(ranks)
  points=0
  num_three=0
  for count in counts.values():
    if count==3:
      num_three+=1
      points+=30
  if points!=0:
    print(num_three,"Three Sages")
  return points

def pairs(ranks): #2 of the same cards
  if len(ranks)<2:
    return 0
  counts=Counter(ranks)
  points=0
  num_pairs=0
  for count in counts.values():
    if count==2:
      num_pairs+=1
      points+=15
  if points!=0:
    print(num_pairs,"Pairs")
  return points

In [ ]:
def score(middle_zone,player_zone):
  ranks=[]
  suits=[]
  for card in middle_zone:
    ranks.append(card[1])
    suits.append(card[0])
  for card in player_zone:
    ranks.append(card[1])
    suits.append(card[0])

  if is_straight(ranks)>0 and is_flush(suits):
    return is_straight(ranks)+is_flush(suits)
  elif is_four_knights(ranks)>0:
    points= is_four_knights(ranks)
    if is_flush(suits)>0:
      points+=is_flush(suits)
    return points
  elif is_flush(suits)>0:
    points=is_flush(suits)
    if is_full_house(ranks):
      points+=is_full_house(ranks)
    elif three_sages(ranks)>0:
      points+=three_sages(ranks)
    elif pairs(ranks)>0:
      points+=pairs(ranks)
    return points
  elif is_straight(ranks)>0:
    return is_straight(ranks)
  elif is_full_house(ranks):
    return is_full_house(ranks)
  elif three_sages(ranks)>0:
    return three_sages(ranks)
  elif pairs(ranks)>0:
    return pairs(ranks)
  return 0


In [ ]:
def main():
  deck=create_deck()

  player_hand=[]
  player_zones=[[],[],[]]
  middle=[[],[],[]]
  trash=[]

  for card in range(5):
    player_hand.append(deck.pop())
  for group in middle:
    for card in range(2):
      group.append(deck.pop())


In [ ]:
score([('Diamonds', '10'), ('Diamonds', '10')],[('Diamonds', '8'), ('Diamonds', 'J'),('Diamonds', 'Q')])

Flush
1 Pairs


80